# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [22]:
# import libraries
import pandas as pd

import numpy as np

from sqlalchemy import create_engine

#For using tokenizer = tokenize in Count Vectorizer
import nltk
nltk.download('punkt')
nltk.download('wordnet')

#NLTK libraries
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

#Machine Learning libraries
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

# Use the below algorithm for improving the model
from sklearn.neighbors import KNeighborsClassifier

# pickle library to save machine learning model
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName3', engine)
print (len(df))

print (len(df['original'].unique()))

# X = df.message.values
# y = df.genre.values

#X = df.iloc[:,1] #Messages as the target variables
X = df.message.values
y = df.iloc[:,4:]# All columns except the first column 


column_names_of_y = list(y.columns)
# print (column_names_of_y)
#y = np.nan_to_num(y) # Filling all the y columns with zero where there is na

df.head()

26207
9631


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
#df.iloc[:,4:]
#df.iloc[:,1]
#X.values.reshape(-1, 1)
print (len(df))



26207


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
        ('vect',CountVectorizer(tokenizer=tokenize)),
        ('tfidf',TfidfTransformer()),
        ('multi',MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y)


# train classifier
pipeline.fit( X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
print (y_test.shape)
print (y_pred.shape)
print(classification_report(y_test,y_pred,target_names=y.columns.values))

# print (y.columns.values)
# for index,category in enumerate(y.columns.values):
#     print(category)
#     print (index)
#     print (classification_report(y_test[:,index],y_pred[:,index]))

(6552, 36)
(6552, 36)
                        precision    recall  f1-score   support

               related       0.78      0.92      0.84      5033
               request       0.40      0.06      0.11      1144
                 offer       0.00      0.00      0.00        40
           aid_related       0.47      0.21      0.29      2766
          medical_help       0.10      0.00      0.00       518
      medical_products       0.07      0.01      0.01       314
     search_and_rescue       0.00      0.00      0.00       185
              security       0.00      0.00      0.00        99
              military       0.00      0.00      0.00       214
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       416
                  food       0.28      0.01      0.02       743
               shelter       0.18      0.01      0.02       591
              clothing       0.67      0.02      0.04        90
                 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [28]:
parameters = {
        'multi__estimator__criterion': ['gini'],
        'multi__estimator__max_depth': [None],
        'multi__estimator__n_estimators': [1]
    }

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=4, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [29]:
# train classifier
cv.fit(X_train, y_train)

# predict on test data
y_pred_model = cv.predict(X_test)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] multi__estimator__criterion=gini, multi__estimator__max_depth=None, multi__estimator__n_estimators=1 
[CV] multi__estimator__criterion=gini, multi__estimator__max_depth=None, multi__estimator__n_estimators=1 
[CV] multi__estimator__criterion=gini, multi__estimator__max_depth=None, multi__estimator__n_estimators=1 
[CV]  multi__estimator__criterion=gini, multi__estimator__max_depth=None, multi__estimator__n_estimators=1, score=0.027930402930402932, total=  52.0s
[CV]  multi__estimator__criterion=gini, multi__estimator__max_depth=None, multi__estimator__n_estimators=1, score=0.02549229125324378, total=  51.7s
[CV]  multi__estimator__criterion=gini, multi__estimator__max_depth=None, multi__estimator__n_estimators=1, score=0.025335775335775336, total=  52.6s


[Parallel(n_jobs=4)]: Done   3 out of   3 | elapsed:  1.3min finished


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [20]:
#### Building Machine Learning Pipeline
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ]))
    ])),

    ('multi', MultiOutputClassifier(KNeighborsClassifier()))
])


#### Training the pipeline
X_train, X_test, y_train, y_test = train_test_split(X, y)

# train classifier
pipeline.fit( X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)


#### Testing the model
print (y_test.shape)
print (y_pred.shape)
print(classification_report(y_test,y_pred,target_names=y.columns.values))

parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'multi__estimator__n_neighbors': [3]
    }

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=4, verbose=3)

(6552, 36)
(6552, 36)
                        precision    recall  f1-score   support

               related       0.78      0.97      0.87      5099
               request       0.35      0.05      0.09      1165
                 offer       0.00      0.00      0.00        35
           aid_related       0.40      0.08      0.13      2749
          medical_help       0.33      0.00      0.00       514
      medical_products       0.00      0.00      0.00       323
     search_and_rescue       0.00      0.00      0.00       178
              security       0.00      0.00      0.00       101
              military       0.00      0.00      0.00       203
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       434
                  food       0.14      0.00      0.01       721
               shelter       0.33      0.00      0.01       608
              clothing       0.00      0.00      0.00       100
                 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [25]:
#To save the model
filepath = ''
#pickle.dump(pipeline, open(filepath, 'wb'))
pickle.dump(pipeline, open('model_pickle', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.